<a href="https://colab.research.google.com/github/python-freak/Bank-Leaving-Problem-using-Keras/blob/master/sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf 
import numpy as np
from scipy.spatial.distance import cdist

In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense , GRU , Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [0]:
import imdb

In [0]:
imdb.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [0]:
x_train_text , y_train = imdb.load_data(train = True)
x_test_text  , y_test  = imdb.load_data(train = False)

In [0]:
print("Train-set size: ",len(x_train_text))
print("Test-set size: " , len(x_test_text))

Train-set size:  25000
Test-set size:  25000


In [0]:
x_train_text[1]

'Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they\'ll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it\'s like to be homeless? That is Goddard Bolt\'s lesson.<br /><br />Mel Brooks (who directs) who stars as Bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival (Jeffery Tambor) to see if he can live in the streets for thirty days withou

In [0]:
y_train[1]

1.0

In [0]:
data_text = x_train_text + x_test_text

In [0]:
num_words = 10000

In [0]:
tokenizer = Tokenizer(num_words = num_words)

In [0]:
%%time
tokenizer.fit_on_texts(data_text)

Wall time: 7.73 s


In [0]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'be': 26,
 'one': 27,
 'he': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'so': 34,
 'who': 35,
 'from': 36,
 'like': 37,
 'or': 38,
 'just': 39,
 'her': 40,
 'out': 41,
 'about': 42,
 'if': 43,
 "it's": 44,
 'has': 45,
 'there': 46,
 'some': 47,
 'what': 48,
 'good': 49,
 'when': 50,
 'more': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'my': 56,
 'even': 57,
 'would': 58,
 'she': 59,
 'which': 60,
 'only': 61,
 'really': 62,
 'see': 63,
 'story': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'me': 68,
 'well': 69,
 'were': 70,
 'than': 71,
 'much': 72,
 'we': 73,
 'bad': 74,
 'been': 75,
 'get': 76,
 'do': 77,
 'great': 78,
 'other': 79,
 'will': 80,
 'also': 81,
 'into': 82,
 'p

In [0]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)

In [0]:
x_train_text[1]

'Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they\'ll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it\'s like to be homeless? That is Goddard Bolt\'s lesson.<br /><br />Mel Brooks (who directs) who stars as Bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival (Jeffery Tambor) to see if he can live in the streets for thirty days withou

In [0]:
np.array(x_train_tokens[1])

array([  38,   14,  744, 3506,   45,   75,   32, 1771,   15,  153,   18,
        110,    3, 1344,    5,  343,  143,   20,    1,  920,   12,   70,
        281, 1228,  395,   35,  115,  267,   36,  166,    5,  368,  158,
         38, 2058,   15,    1,  504,   88,   83,  101,    4,    1, 4339,
         14,   39,    3,  432, 1148,  136, 8697,   42,  177,  138,   14,
       2791,    1,  295,   20, 5276,  351,    5, 3029, 2310,    1,   38,
       8697,   43, 3611,   26,  365,    5,  127,   53,   20,    1, 2032,
          7,    7,   18,   48,   43,   22,   70,  358,    3, 2343,    5,
        420,   20,    1, 2032,   15,    3, 3346,  208,    1,   22,  281,
         66,   36,    3,  344,    1,  728,  730,    3, 3864, 1320,   20,
          1, 1543,    3, 1293,    2,  267,   22,  281, 2734,    5,   63,
         48,   44,   37,    5,   26, 4339,   12,    6, 2079,    7,    7,
       3425, 2891,   35, 4446,   35,  405,   14,  297,    3,  986,  128,
         35,   45,  267,    8,    1,  181,  366, 69

In [0]:
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

In [0]:
len(x_train_tokens)

25000

In [0]:
for tokens in x_train_tokens:
    print(len(tokens))
    print('\n')

127


401


134


119


119


174


110


307


401


317


277


85


255


222


142


156


185


294


127


152


206


550


166


160


138


99


106


296


150


135


239


118


162


161


429


468


54


292


216


316


345


156


526


237


109


317


916


489


87


1745


429


516


178


523


201


681


171


159


52


268


97


159


131


218


305


151


102


205


105


87


111


177


278


313


42


141


106


313


72


217


114


241


106


116


55


55


253


123


169


152


245


77


193


118


166


53


59


85


122


165


158


201


105


144


541


162


255


233


139


475


233


310


189


54


160


117


233


347


256


316


173


80


225


154


135


214


490


151


105


103


151


140


205


142


298


171


141


201


604


229


299


195


317


130


299


101


132


153


116


595


145


162


386


282


488


277


360


151


143


180


187


128


134


281


481


86


367


328


151


150



593


101


195


224


130


106


715


160


139


54


157


127


216


94


538


630


215


162


162


388


862


574


290


473


193


68


154


242


109


81


136


274


142


75


109


232


126


199


147


310


126


303


91


154


117


123


232


158


825


636


256


323


260


541


207


122


173


116


648


489


121


104


241


132


140


107


175


187


411


486


112


168


104


123


146


80


34


254


375


154


145


329


156


591


256


225


132


186


225


383


336


97


150


149


108


47


119


71


165


443


159


265


728


424


905


145


194


119


423


141


133


170


139


143


28


128


315


50


48


56


121


143


122


319


148


290


80


55


127


150


366


117


57


201


175


159


471


476


397


204


207


134


206


380


189


348


137


226


716


194


147


312


192


53


226


745


262


270


125


418


197


510


119


163


167


177


373


412


380


151

160


404


260


251


411


542


289


172


104


91


180


315


476


192


174


473


62


194


151


127


121


545


92


32


125


39


136


189


163


124


271


48


143


153


119


101


222


111


158


132


281


158


170


203


217


212


481


210


150


197


320


264


213


98


134


266


448


173


150


193


208


144


601


622


159


119


142


169


282


179


123


172


216


121


165


256


106


148


256


490


136


351


100


121


145


113


1076


122


52


117


133


347


119


155


244


357


149


165


89


575


138


286


193


229


187


130


38


126


132


193


276


61


142


82


68


87


67


457


116


158


101


238


59


245


147


123


142


198


190


209


253


240


88


231


191


289


323


254


96


172


147


194


111


107


197


120


306


178


295


477


365


724


292


553


228


424


86


112


100


153


193


101


42


30


41


333


22


210


130


57


42




40


290


191


348


79


290


286


208


113


146


263


460


647


555


97


324


211


442


161


166


490


304


167


144


149


375


155


352


41


107


234


234


269


505


94


105


143


154


169


161


420


111


200


214


183


63


64


36


105


51


83


79


166


176


130


128


110


133


114


112


193


213


171


55


235


76


80


67


162


55


140


116


52


39


50


58


215


199


45


33


64


69


115


49


159


309


140


262


98


217


77


125


134


565


81


154


265


442


179


202


276


270


320


142


139


276


109


200


285


43


308


651


558


325


121


142


168


99


120


289


214


143


123


114


114


109


44


104


157


239


50


103


92


348


169


114


109


158


203


145


188


114


560


234


441


530


346


450


113


119


133


146


183


278


142


133


197


131


168


304


127


353


41


115


68


838


266


491


233


127


117


452


4

106


183


358


285


265


284


182


190


174


207


74


138


136


113


121


121


185


145


115


210


153


581


118


150


133


193


594


299


243


585


132


94


249


194


115


142


154


422


130


182


103


169


197


189


150


152


230


318


171


112


293


505


149


238


189


173


356


195


240


105


262


300


99


131


267


238


173


236


539


148


62


213


103


138


108


152


192


128


214


300


369


569


258


116


206


230


264


150


187


134


223


318


197


123


287


148


788


151


130


139


117


592


219


126


196


157


560


126


118


94


137


487


124


80


78


122


848


116


204


65


201


104


279


76


46


171


128


249


245


271


143


568


99


225


131


213


230


168


536


601


400


238


159


211


47


153


114


129


235


109


449


116


115


293


126


104


176


346


162


128


161


125


134


120


181


434


298


111


75



...



107


292


410


121


141


189


124


129


114


207


122


569


139


107


110


140


266


142


238


98


111


137


220


709


60


189


379


338


139


109


828


103


550


84


315


163


146


228


252


223


129


148


151


156


163


185


186


181


383


239


700


280


897


315


331


120


115


118


284


273


194


141


166


972


131


159


131


189


185


113


564


129


161


190


82


150


160


465


157


556


427


91


330


333


121


119


136


330


149


321


160


541


125


274


219


190


137


114


36


189


305


69


95


100


151


85


736


342


81


44


49


194


129


193


469


256


116


175


263


235


108


185


52


120


69


329


446


255


59


264


88


79


265


227


95


123


131


211


151


143


340


316


185


146


160


139


121


359


247


120


156


127


173


126


265


139


124


96


339


204


130


178


177


174


288


531


587


325


959


319

169


329


64


158


31


65


290


167


111


116


307


118


291


109


120


238


595


406


168


352


114


141


196


136


118


482


114


275


232


246


406


586


254


207


59


104


265


148


238


253


162


271


789


130


949


283


498


184


129


119


115


149


176


146


45


153


211


144


362


140


164


136


387


123


219


232


225


63


106


149


137


322


227


524


128


217


10


108


33


187


131


196


129


154


130


153


67


676


120


467


215


124


205


220


96


223


380


146


153


147


160


189


368


129


370


149


178


135


145


229


388


126


118


148


75


58


242


223


393


783


142


303


624


578


307


109


30


136


101


101


56


69


187


104


44


168


184


169


202


169


115


166


252


225


671


112


94


94


276


371


94


167


179


229


167


56


134


134


195


128


228


124


169


137


108


281


130


689


371


288






125


58


347


35


156


173


483


127


180


153


91


121


107


242


61


151


569


196


281


405


132


129


47


146


52


237


236


105


196


596


433


519


113


173


205


251


194


118


165


121


193


187


462


79


221


268


219


174


167


122


145


364


236


208


241


163


149


58


254


135


132


462


124


130


354


129


167


139


938


484


128


293


149


205


119


222


244


366


632


200


100


118


171


129


105


128


502


225


110


343


889


353


121


244


328


208


138


130


578


233


151


136


134


265


242


57


327


501


165


751


125


64


541


179


296


112


306


111


270


110


131


141


143


145


220


69


133


338


136


553


164


170


702


348


114


254


59


287


56


90


163


185


154


220


601


359


271


141


234


148


119


166


189


41


595


96


101


112


161


177


150


163


256


121


81


156


107


188


183


3

374


186


203


117


468


187


45


142


584


45


222


108


150


194


400


42


267


233


163


161


140


197


138


106


191


381


176


53


42


232


113


159


183


196


106


229


157


206


127


145


304


645


324


167


110


130


180


143


204


193


179


335


163


117


69


226


148


118


254


193


56


101


147


190


150


527


115


115


186


549


109


672


125


98


148


153


130


154


163


268


230


133


207


115


208


113


74


249


108


350


307


708


53


118


132


154


147


119


227


188


111


182


106


142


98


55


278


211


109


595


156


132


193


129


127


304


254


258


115


161


154


169


435


304


223


50


118


171


347


102


170


45


169


218


389


273


212


121


251


209


669


124


299


235


323


143


173


296


150


149


204


182


238


128


252


119


149


109


199


125


147


647


119


465


528


41


115


114


163


5



88


111


117


551


98


83


206


215


214


88


164


175


763


100


88


282


123


334


326


513


69


352


130


38


165


122


497


227


258


160


196


210


130


956


542


220


191


40


64


182


338


807


85


198


192


378


214


132


242


407


214


95


693


56


394


69


283


127


109


49


152


119


48


96


236


147


164


130


186


152


159


144


116


167


162


316


92


132


384


100


161


60


196


118


46


218


228


454


163


144


151


243


121


123


138


133


203


126


133


125


244


118


109


157


415


354


133


102


127


149


117


265


167


145


138


107


256


386


394


757


192


157


780


187


550


67


131


202


652


168


272


517


164


321


143


154


364


114


149


223


140


118


121


136


131


118


123


325


87


131


380


158


121


64


179


379


206


87


143


181


160


167


225


181


169


256


531


123


155


247


1

In [0]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
num_tokens

array([127, 401, 134, ..., 253, 119, 352])

In [0]:
num_tokens.max()

2209

In [0]:
mean = num_tokens.mean()

In [0]:
std = num_tokens.std()

In [0]:
max_tokens = mean + 2*std
type(max_tokens)
max_tokens = int(max_tokens)
max_tokens

544

In [0]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9453

In [0]:
pad = 'pre'

In [0]:
x_train_pad = pad_sequences(x_train_tokens , maxlen = max_tokens , padding = pad , truncating = pad)

In [0]:
x_train_pad[50]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,   10,  165,    9,   34,  491,   12,
         57,  100,   29,  132,  153,   73,   23,  130,  681,   42,   11,
         17,  528,   11,   17,  292,   12,   74,   38,  329,   83,  589,
         57, 1259,    5,  705,   42,    9,   10, 12

In [0]:
x_test_pad = pad_sequences(x_test_tokens , maxlen = max_tokens , padding = pad , truncating = pad)

In [0]:
x_test_pad[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [0]:
type(tokenizer.word_index)

dict

In [0]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values() , idx.keys()))
inverse_map

{1: 'the',
 2: 'and',
 3: 'a',
 4: 'of',
 5: 'to',
 6: 'is',
 7: 'br',
 8: 'in',
 9: 'it',
 10: 'i',
 11: 'this',
 12: 'that',
 13: 'was',
 14: 'as',
 15: 'for',
 16: 'with',
 17: 'movie',
 18: 'but',
 19: 'film',
 20: 'on',
 21: 'not',
 22: 'you',
 23: 'are',
 24: 'his',
 25: 'have',
 26: 'be',
 27: 'one',
 28: 'he',
 29: 'all',
 30: 'at',
 31: 'by',
 32: 'an',
 33: 'they',
 34: 'so',
 35: 'who',
 36: 'from',
 37: 'like',
 38: 'or',
 39: 'just',
 40: 'her',
 41: 'out',
 42: 'about',
 43: 'if',
 44: "it's",
 45: 'has',
 46: 'there',
 47: 'some',
 48: 'what',
 49: 'good',
 50: 'when',
 51: 'more',
 52: 'very',
 53: 'up',
 54: 'no',
 55: 'time',
 56: 'my',
 57: 'even',
 58: 'would',
 59: 'she',
 60: 'which',
 61: 'only',
 62: 'really',
 63: 'see',
 64: 'story',
 65: 'their',
 66: 'had',
 67: 'can',
 68: 'me',
 69: 'well',
 70: 'were',
 71: 'than',
 72: 'much',
 73: 'we',
 74: 'bad',
 75: 'been',
 76: 'get',
 77: 'do',
 78: 'great',
 79: 'other',
 80: 'will',
 81: 'also',
 82: 'into',
 83

In [0]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens]
    text = " ".join(words)
    return text

In [0]:
x_train_text[4]

'This is not the typical Mel Brooks film. It was much less slapstick than most of his movies and actually had a plot that was followable. Leslie Ann Warren made the movie, she is such a fantastic, under-rated actress. There were some moments that could have been fleshed out a bit more, and some scenes that could probably have been cut to make the room to do so, but all in all, this is worth the price to rent and see it. The acting was good overall, Brooks himself did a good job without his characteristic speaking to directly to the audience. Again, Warren was the best actor in the movie, but "Fume" and "Sailor" both played their parts well.'

In [0]:
tokens_to_string(x_train_tokens[4])

'this is not the typical mel brooks film it was much less slapstick than most of his movies and actually had a plot that was leslie ann warren made the movie she is such a fantastic under rated actress there were some moments that could have been fleshed out a bit more and some scenes that could probably have been cut to make the room to do so but all in all this is worth the price to rent and see it the acting was good overall brooks himself did a good job without his characteristic speaking to directly to the audience again warren was the best actor in the movie but and sailor both played their parts well'

In [0]:
model = Sequential()

In [0]:
embedding_size = 32 

In [0]:
model.add(Embedding(input_dim = num_words , output_dim = embedding_size , input_length = max_tokens , name = "layer_embedding1"))

In [0]:
model.add(GRU(units = 16 , return_sequences = True))

In [0]:
model.add(GRU(units = 8,return_sequences = True))

In [0]:
model.add(GRU(units = 4))

In [0]:
model.add(Dense (1 , activation = 'sigmoid'))

In [0]:
optimizer = Adam(lr = 1e-3)

In [0]:
model.compile(loss = 'binary_crossentropy' , optimizer = optimizer , metrics = ['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding1 (Embedding) (None, 544, 32)           320000    
_________________________________________________________________
gru_3 (GRU)                  (None, 544, 32)           6240      
_________________________________________________________________
gru_4 (GRU)                  (None, 544, 16)           2352      
_________________________________________________________________
gru_5 (GRU)                  (None, 8)                 600       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 329,201
Trainable params: 329,201
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(x_train_pad , y_train , validation_split = 0.05 , epochs = 15 , batch_size = 128)

Train on 23750 samples, validate on 1250 samples
Epoch 1/15
23750/23750 [==============================] - 133s 6ms/step - loss: 0.5126 - acc: 0.7272 - val_loss: 0.2960 - val_acc: 0.8808
Epoch 2/15
23750/23750 [==============================] - 138s 6ms/step - loss: 0.2806 - acc: 0.8907 - val_loss: 0.3034 - val_acc: 0.8696
Epoch 3/15
23750/23750 [==============================] - 160s 7ms/step - loss: 0.2119 - acc: 0.9240 - val_loss: 0.3834 - val_acc: 0.8328
Epoch 4/15
23750/23750 [==============================] - 134s 6ms/step - loss: 0.1714 - acc: 0.9422 - val_loss: 0.4152 - val_acc: 0.8328
Epoch 5/15
23750/23750 [==============================] - 133s 6ms/step - loss: 0.1473 - acc: 0.9522 - val_loss: 0.4292 - val_acc: 0.8400
Epoch 6/15
23750/23750 [==============================] - 146s 6ms/step - loss: 0.1212 - acc: 0.9621 - val_loss: 0.3854 - val_acc: 0.8624
Epoch 7/15
23750/23750 [==============================] - 135s 6ms/step - loss: 0.1032 - acc: 0.9693 - val_loss: 0.4248 - v

In [0]:
result = model.evaluate(x_test_pad , y_test)

25000/25000 [==============================] - 74s 3ms/step


In [0]:
print("Accuracy: {0: .2%}".format(result[1]))

Accuracy:  84.84%
